In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import wandb as wb
from src.MetaHeauristic import WandbHandler, OBP
import re, logging
import time

In [2]:
wb.login()

wandb: Currently logged in as: glasshan914 (glasshan9140) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
folder_list=glob.glob('E:/glass_git/AI-portfolio/order_batch_picking/data/Orderlinelist/*.txt')
dist_matrix=pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')
dist_matrix=dist_matrix.to_numpy()

datasets=[]
for file in folder_list:
    df=pd.read_csv(file,sep='\t',header=None)
    cols=['order_num','index','rack','bay','tier','slot']
    df.columns=cols
    datasets.append(df)
print(f'total files: {len(datasets)}')

total files: 959


In [4]:
#여러개 파일 중에 N번째 파일 선택
f=np.random.random_integers(1,len(datasets),1).item() #몇번째 파일을 선택할 것인지
dataset=datasets[f]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5972\2973922749.py:2: DeprecationWarning: This function is deprecated. Please call randint(1, 959 + 1) instead
  f=np.random.random_integers(1,len(datasets),1).item() #몇번째 파일을 선택할 것인지


In [5]:
def get_sample(dataset)->pd.DataFrame:
    total_order_num=len(dataset.order_num.unique())
    #파일 중 일부 order만 샘플링 후 수행
    sample_order_num=np.random.random_integers(1,total_order_num,1) #N: 몇개의 order를 뽑을 것인지
    sample_order=np.random.choice(total_order_num,size=sample_order_num,replace=False) #선택된 order_id
    sample1=dataset[dataset['order_num'].isin(sample_order)] 
    print(f'{f}번째 파일, 주문 수: {sample_order_num}')
    print(f'주문 id: {sample_order}')
    return sample1, f, sample_order_num, sample_order

In [6]:
sample,f,sample_order_num,sample_order=get_sample(dataset)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5972\1098011882.py:4: DeprecationWarning: This function is deprecated. Please call randint(1, 1002 + 1) instead
  sample_order_num=np.random.random_integers(1,total_order_num,1) #N: 몇개의 order를 뽑을 것인지


907번째 파일, 주문 수: [391]
주문 id: [ 509  668   15   18  479  955  124  365  285  127  437  162  952  699
  306  718  109  837   89  446   52  316  233  146  419  538  258  844
  673  873  943  219  357  248   21  959  449   88  497   33   72  129
  682  223  377  406  701   74  159  243  606  708  346  547  941  704
  382  304  577  350  849  568  433  661  355  734  812  818  345  733
  543  766  154  582  931  707  732   32  232  868  675  183  609  474
  366  203  289  113  308  324  409  514  642  483  647    3   96  775
   56  702  505  117  240  544  492  912  347  649  164  759  872  966
  806  515  245  426  136  326  293  621  898  138  692  631  194  229
  683   95  451  506    6  590  381  879  928  660  488  265  295   24
  221  693  949  522  995  470  622  963  387  386  858  712  318   65
  494  700  626  604  261  147  897  527  788  216  555  801  845  691
  672  554  585  960  726  772  651  911  374  802  918  541  598   11
  131  104  267  337  325  964  438  108  478   

In [ ]:
logger = logging.getLogger("mealpy")
logger.setLevel(logging.INFO)
logger.propagate = False  # 중복 출력 방지 (필요 시)
if not any(isinstance(h, WandbHandler) for h in logger.handlers):
    logger.addHandler(WandbHandler())

    
location=pd.read_csv('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/location.csv')
dist_matrix=pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')
dist_matrix=dist_matrix.to_numpy()

try_id=time.strftime('%y%m%d_%H%M')
K = 10
ALPHA = 0.8
model_params={'epoch':1000 , 'popsize': 100, 'nlimits': 3 , 'seed': 42} 

wbrun=wb.init(project="OBP", name=f"abc-{try_id}", 
        config=dict(
            algo="ABC", 
            epoch=model_params.get('epoch'), 
            pop_size=model_params.get('popsize'), 
            n_limits=model_params.get('nlimits'), 
            seed=model_params.get('seed'),
            data_file_num=f,
            the_number_of_orders=sample_order_num,
            orders_id=sample_order,
            job=K,
            alpha=ALPHA,
            depot=0
))

with wbrun:
    obp=OBP(dist_matrix,sample,location, K,ALPHA)
    model=obp.model(model_params)
    best=obp.run(model)

In [ ]:
save_path=f'E:/glass_git/AI-portfolio/order_batch_picking/result/{wbrun.name}.png'
obp.result(best,location,save_path)

In [ ]:
Model_list=['AEO','WaOA','ServalOA','STO']

for model_name in Model_list:
    try_id=time.strftime('%y%m%d_%H%M')
    K = 10
    ALPHA = 0.8
    model_params={'epoch':1000 , 'popsize': 100, 'nlimits': 3 , 'seed': 42} 

    wbrun=wb.init(project="OBP", name=f"{model_name}-{try_id}", 
            config=dict(
                algo=model_name, 
                epoch=model_params.get('epoch'), 
                pop_size=model_params.get('popsize'), 
                n_limits=model_params.get('nlimits'), 
                seed=model_params.get('seed'),
                data_file_num=f,
                the_number_of_orders=sample_order_num,
                orders_id=sample_order,
                job=K,
                alpha=ALPHA,
                depot=0
    ))

    with wbrun:
        obp=OBP(model_name,dist_matrix,sample,location, K,ALPHA)
        model=obp.model(model_params)
        best=obp.run(model)